# Crypto Forecasting 

In [14]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json

%matplotlib inline

# Stocks Data

In [15]:
# Load .env environment variables
load_dotenv("alpaca.env")

True

In [25]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [26]:
# Set start and end dates of five years back from today.
start_date = pd.Timestamp("2018-05-01", tz = "America/New_York").isoformat()
end_date = pd.Timestamp("2023-05-01", tz = "America/New_York").isoformat()

# Set the tickers
tickers = ["AAPL", "MSFT", "AMD"]
timeframe = "1D"

# Get current closing prices 
stocks_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
AAPL = stocks_df[stocks_df["symbol"]=="AAPL"].drop("symbol", axis=1)
MSFT = stocks_df[stocks_df["symbol"]=="MSFT"].drop("symbol", axis=1)
AMD = stocks_df[stocks_df["symbol"]=="AMD"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
stocks_df = pd.concat([AAPL, MSFT, AMD],axis=1, keys=["AAPL","MSFT", "AMD"])

# Preview DataFrame
stocks_df

AAPL                                       \
                               open     high       low   close    volume   
timestamp                                                                  
2018-05-01 04:00:00+00:00  166.4102  169.200  165.2700  169.10  53569474   
2018-05-02 04:00:00+00:00  175.2250  177.750  173.8000  176.57  66851451   
2018-05-03 04:00:00+00:00  175.8800  177.500  174.4411  176.89  34068446   
2018-05-04 04:00:00+00:00  178.2500  184.250  178.1700  183.83  56204195   
2018-05-07 04:00:00+00:00  185.1800  187.670  184.7500  185.16  42613114   
...                             ...      ...       ...     ...       ...   
2023-04-25 04:00:00+00:00  165.1900  166.305  163.7300  163.77  48849358   
2023-04-26 04:00:00+00:00  163.9000  165.280  162.8000  163.76  45524694   
2023-04-27 04:00:00+00:00  165.1900  168.560  165.1900  168.41  64891617   
2023-04-28 04:00:00+00:00  168.4900  169.850  167.8801  169.68  53888026   
2023-05-01 04:00:00+00:00  169.2800  170.450  168.6400  169.59  52546063   

                                                     MSFT                   \
                          trade_count        vwap    open     high     low   
timestamp                                                                    
2018-05-01 04:00:00+00:00      353863  169.130276   93.21   95.290   92.79   
2018-05-02 04:00:00+00:00      437602  176.126868   94.99   95.170   93.19   
2018-05-03 04:00:00+00:00      246062  176.100682   92.96   94.925   92.45   
2018-05-04 04:00:00+00:00      370771  182.554203   93.32   95.370   92.92   
2018-05-07 04:00:00+00:00      267449  186.216900   95.17   96.710   95.10   
...                               ...         ...     ...      ...     ...   
2023-04-25 04:00:00+00:00      502773  164.647271  279.51  281.600  275.37   
2023-04-26 04:00:00+00:00      494174  163.908156  296.70  299.570  292.73   
2023-04-27 04:00:00+00:00      575890  167.319460  295.97  305.200  295.25   
2023-04-28 04:00:00+00:00      513328  168.948227  304.01  308.930  303.31   
2023-05-01 04:00:00+00:00      541577  169.657746  306.97  308.600  305.15   

                           ...                                       AMD  \
                           ...    volume trade_count        vwap    open   
timestamp                  ...                                             
2018-05-01 04:00:00+00:00  ...  31409017      209726   94.199608  10.830   
2018-05-02 04:00:00+00:00  ...  27471134      188110   94.016453  11.100   
2018-05-03 04:00:00+00:00  ...  31142548      238268   93.702789  10.902   
2018-05-04 04:00:00+00:00  ...  22531383      165938   94.745049  10.880   
2018-05-07 04:00:00+00:00  ...  24262669      172194   96.187561  11.320   
...                        ...       ...         ...         ...     ...   
2023-04-25 04:00:00+00:00  ...  47004633      627981  280.958848  86.900   
2023-04-26 04:00:00+00:00  ...  64681706      869828  296.027735  86.510   
2023-04-27 04:00:00+00:00  ...  46463058      650724  302.549963  86.410   
2023-04-28 04:00:00+00:00  ...  31737282      485116  306.123532  87.020   
2023-05-01 04:00:00+00:00  ...  21329194      317112  306.613708  91.030   

                                                                         \
                             high      low  close    volume trade_count   
timestamp                                                                 
2018-05-01 04:00:00+00:00  11.150  10.7700  11.13  43441602       90980   
2018-05-02 04:00:00+00:00  11.155  10.9700  10.97  29267466       77517   
2018-05-03 04:00:00+00:00  11.060  10.8300  10.93  33310596       89301   
2018-05-04 04:00:00+00:00  11.290  10.8701  11.28  47885590      120350   
2018-05-07 04:00:00+00:00  11.680  11.3000  11.59  54003947      132612   
...                           ...      ...    ...       ...         ...   
2023-04-25 04:00:00+00:00  87.060  83.7600  83.80  52893533      342477   
2023-04-26 04:00:00+00:00  87.350  85.3200  85.94  4781519

In [98]:
# Count null
stocks_df.isnull().sum()

AAPL  open           0
      high           0
      low            0
      close          0
      volume         0
      trade_count    0
      vwap           0
MSFT  open           0
      high           0
      low            0
      close          0
      volume         0
      trade_count    0
      vwap           0
AMD   open           0
      high           0
      low            0
      close          0
      volume         0
      trade_count    0
      vwap           0
dtype: int64

# Forex Data

In [49]:
from pathlib import Path

In [91]:
# Reading EUR
EUR_path = Path("../resources/EURUSD.csv")
EUR_df = pd.read_csv(EUR_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
EUR_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-04-27,1.36015,1.36820,1.35850,1.36235,1023126
2007-04-28,1.36235,1.36540,1.36155,1.36520,95014
2007-04-30,1.36305,1.36777,1.35884,1.36581,545597
2007-05-01,1.36586,1.36711,1.35897,1.35984,664031
2007-05-02,1.35986,1.36210,1.35598,1.35962,659075


In [61]:
# Reading GBP
GBP_path = Path("../resources/GBPUSD.csv")
GBP_df = pd.read_csv(GBP_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
GBP_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-04-26,1.99140,1.99180,1.9897,1.9901,18697
2007-04-27,1.99010,2.00430,1.9864,1.9984,492786
2007-04-28,1.99840,1.99950,1.9954,1.9976,90222
2007-04-30,1.99578,2.00310,1.9890,1.9998,426199
2007-05-01,1.99980,2.00745,1.9980,1.9991,601529


In [60]:
# Reading JPY
JPY_path = Path("../resources/JPYUSD.csv")
JPY_df = pd.read_csv(JPY_path, index_col = "Date", parse_dates = True, infer_datetime_format = True)
JPY_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-04-06,118.830,119.390,118.725,119.340,157979
2007-04-07,119.340,119.360,119.210,119.245,10085
2007-04-09,119.245,119.380,119.175,119.280,212315
2007-04-10,119.280,119.385,118.760,119.065,700242
2007-04-11,119.070,119.520,118.870,119.360,753655


In [81]:
# Concatenate the DataFrames
forex_df = pd.concat([EUR_df, GBP_df, JPY_df],axis=1, keys=["EUR","GBP", "JPY"])

# Preview DataFrame
forex_df

EUR                                           GBP           \
               Open     High      Low    Close    Volume     Open     High   
Date                                                                         
2007-04-06      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-07      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-09      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-10      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
2007-04-11      NaN      NaN      NaN      NaN       NaN      NaN      NaN   
...             ...      ...      ...      ...       ...      ...      ...   
2023-04-29  1.10394  1.10406  1.10119  1.10144   48265.0  1.25801  1.25837   
2023-05-01  1.10240  1.10355  1.09661  1.09692  209835.0  1.25538  1.25689   
2023-05-02  1.09692  1.10066  1.09422  1.09965  354234.0  1.24929  1.25120   
2023-05-03  1.09964  1.10593  1.09940  1.10478  281057.0  1.24730  1.25531   
2023-05-04  1.10478  1.10916  1.10371  1.10679  166874.0  1.25446  1.25917   

                                            JPY                             \
                Low    Close    Volume     Open     High      Low    Close   
Date                                                                         
2007-04-06      NaN      NaN       NaN  118.830  119.390  118.725  119.340   
2007-04-07      NaN      NaN       NaN  119.340  119.360  119.210  119.245   
2007-04-09      NaN      NaN       NaN  119.245  119.380  119.175  119.280   
2007-04-10      NaN      NaN       NaN  119.280  119.385  118.760  119.065   
2007-04-11      NaN      NaN       NaN  119.070  119.520  118.870  119.360   
...             ...      ...       ...      ...      ...      ...      ...   
2023-04-29  1.25572  1.25608   20129.0  135.995  136.348  135.960  136.243   
2023-05-01  1.24863  1.24929  129575.0  136.105  137.409  136.105  137.354   
2023-05-02  1.24352  1.24734  144194.0  137.354  137.772  136.387  136.464   
2023-05-03  1.24629  1.25446  123853.0  136.464  136.700  135.067  135.312   
2023-05-04  1.25374  1.25840  135690.0  135.313  135.704  134.345  134.537   

                      
              Volume  
Date                  
2007-04-06  157979.0  
2007-04-07   10085.0  
2007-04-09  212315.0  
2007-04-10  700242.0  
2007-04-11  753655.0  
...              ...  
2023-04-29   70947.0  
2023-05-01  430547.0  
2023-05-02  607095.0  
2023-05-03  551343.0  
2023-05-04  307813.0  

[5029 rows x 15 columns]

In [103]:
# Choose the desired timeframe
forex_df = forex_df.loc["2018-05-01":"2023-05-01"]
forex_df

EUR                                           GBP           \
               Open     High      Low    Close    Volume     Open     High   
Date                                                                         
2018-05-01  1.20735  1.21062  1.19814  1.19995  208172.0  1.37496  1.37765   
2018-05-02  1.19994  1.20316  1.19531  1.19672  245713.0  1.36091  1.36656   
2018-05-03  1.19674  1.20255  1.19378  1.19706  280127.0  1.36084  1.36519   
2018-05-04  1.19704  1.20041  1.19107  1.19452  235191.0  1.35681  1.35856   
2018-05-05  1.19451  1.19659  1.19365  1.19550   42242.0  1.35337  1.35522   
...             ...      ...      ...      ...       ...      ...      ...   
2023-04-26  1.09685  1.10952  1.09641  1.10509  395863.0  1.23951  1.25150   
2023-04-27  1.10509  1.10634  1.09920  1.10154  358245.0  1.24837  1.24992   
2023-04-28  1.10156  1.10449  1.09624  1.10394  385123.0  1.24692  1.25804   
2023-04-29  1.10394  1.10406  1.10119  1.10144   48265.0  1.25801  1.25837   
2023-05-01  1.10240  1.10355  1.09661  1.09692  209835.0  1.25538  1.25689   

                                            JPY                             \
                Low    Close    Volume     Open     High      Low    Close   
Date                                                                         
2018-05-01  1.35880  1.36092  118028.0  109.287  109.794  109.086  109.686   
2018-05-02  1.35804  1.36080  144116.0  109.686  110.036  109.515  109.903   
2018-05-03  1.35377  1.35684  185202.0  109.902  109.963  108.929  109.048   
2018-05-04  1.34861  1.35338  139486.0  109.049  109.272  108.648  109.166   
2018-05-05  1.35232  1.35253   25412.0  109.167  109.202  108.957  109.101   
...             ...      ...       ...      ...      ...      ...      ...   
2023-04-26  1.23908  1.24836  145123.0  133.967  134.014  133.013  133.722   
2023-04-27  1.24359  1.24688  158894.0  133.723  134.197  133.204  134.054   
2023-04-28  1.24462  1.25803  184504.0  134.052  136.560  133.335  135.994   
2023-04-29  1.25572  1.25608   20129.0  135.995  136.348  135.960  136.243   
2023-05-01  1.24863  1.24929  129575.0  136.105  137.409  136.105  137.354   

                      
              Volume  
Date                  
2018-05-01  102630.0  
2018-05-02  107271.0  
2018-05-03  131190.0  
2018-05-04  137849.0  
2018-05-05   18969.0  
...              ...  
2023-04-26  714287.0  
2023-04-27  587795.0  
2023-04-28  601996.0  
2023-04-29   70947.0  
2023-05-01  430547.0  

[1564 rows x 15 columns]

In [104]:
# Count null
forex_df.isnull().sum()

EUR  Open      0
     High      0
     Low       0
     Close     0
     Volume    0
GBP  Open      0
     High      0
     Low       0
     Close     0
     Volume    0
JPY  Open      0
     High      0
     Low       0
     Close     0
     Volume    0
dtype: int64